In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf
from transformer import Transformer

In [ ]:
transformer = Transformer(vocab_size=9000,
                          d_model=128,
                          num_layers=4,
                          num_heads=4,
                          d_ff=512,
                          dropout=0.3)

[모델 빌드]
인풋 모양은 (2(인코더 입력, 디코더 입력), 배치 사이, d_model)을 의미

In [ ]:
transformer.build(input_shape=(2, 1, 9000))

[손실 함수 정의]
예제는 다중 클래스 분류 문제. 이때 레이블이 정수 형태이므로 손실 함수는 SparseCategoricalCrossentropy 사용

In [7]:
def loss_function(ans, pred):
    """
    다중 클래스 분류 문제를 위한 손실 함수 정의
    
    :param ans: 해당 데이터의 실제 정답
    :param pred: 모델이 생성해낸 예측 레이블
    :return: 손실값
    """
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(ans, pred)
    mask = tf.cast(tf.not_equal(ans, 0), tf.float32)
    loss = tf.multiply(loss, mask)
    
    return tf.reduce_mean(loss)

[데이터 로드]
챗봇 데이터를 로드
학습 기반 토크나이저 사용을 위해 구두점 처리

In [8]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('CHatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [9]:
print(f'샘플의 개수 : {len(train_data)}')

샘플의 개수 : 11823


In [10]:
print(train_data.isnull().sum())

Q        0
A        0
label    0
dtype: int64


In [16]:
# 구두점 제거 대신 띄어쓰기를 추가하여 다른 문자와 구분
# 정규식 사용하여 처리
questions = []
for sentence in train_data['Q']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)
    
answers = []
for sentence in train_data['A']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [17]:
print(questions[:5])
print(answers[:5])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']
